In [142]:
pathToTestCases = "../test"
pathToBuild = "../build"
pathToTestPre = "../testPre"

In [143]:
class SolveRes:
    def __init__(self, ret, name):
        self.log = ret
        self.name = name
        self.res = ret[-2].split()[-1]
        self.time = ret[-1].split()[2]
    def prettyPrint(self):
        print("Solved problem:", self.name)
        print("Result:", self.res)
        print("Time:", self.time)

In [144]:
class PresolveRes:
    def __init__(self, preRet, name):
        retLen = len(preRet) 
        self.name = name
        self.log = preRet
        self.status = preRet[-2].split()[-1]
        self.nrows = preRet[-5].split()[-1]
        self.ncols = preRet[-4].split()[1]
        self.time = preRet[-1].split()[3]
    def prettyPrint(self):
        print("Presolved problem:", self.name)
        print("nrows:", self.nrows)
        print("ncols:", self.ncols)
        print("Status:", self.status)
        print("Time:", self.time)

In [181]:
 class ComparePreAndBase:
    def __init__(self, res_base, res_withpre, stats_presolver, filename):
        self.name = filename
        self.res_base = res_base
        self.res_withpre = res_withpre
        self.stats_presolver = stats_presolver

In [145]:
def presolve(filename):
    ret = !time {pathToBuild}/PresolveOpb {pathToTestCases}/{filename} {pathToTestPre}/pre.{filename}
    return PresolveRes(ret, filename)

In [146]:
def solve(filename, to_pre_or_not_to_pre):
    if(to_pre_or_not_to_pre):
        ret = !time {pathToBuild}/roundingsat {pathToTestPre}/pre.{filename}
    else:
        ret = !time {pathToBuild}/roundingsat {pathToTestCases}/{filename}
    return SolveRes(ret, filename)

In [182]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)



In [147]:
alltestfiles = !ls ../test

In [154]:
quickProblems = ['air01.0.s.opb',
 'air01.0.u.opb',
 'air02.0.s.opb',
 'air02.0.u.opb',
 'bm23.0.s.opb',
 'bm23.0.u.opb',
 'diamond.0.d.opb',
 'p0040.0.s.opb',
 'p0040.0.u.opb',
 'p0291.0.s.opb',
 'p0291.0.u.opb',
 'pipex.0.s.opb',
 'pipex.0.u.opb',
 'sentoy.0.s.opb',
 'sentoy.0.u.opb',
 'stein15.0.s.opb',
 'stein15.0.u.opb',
 'stein9.0.s.opb',
 'stein9.0.u.opb']




In [ ]:
results = []

In [178]:
for prob in quickProblems:
    try:
        with time_limit(10):
            solveres = solve(prob, False)
        except TimeoutException as e:
            print("Timed out solving (no presolver): " + prob)
    
    presolveinf = presolve(prob)
    try:
        with time_limit(10):
            solveres = solve(prob, False)
        except TimeoutException as e:
            print("Timed out solving (no presolver): " + prob)
    solverespre = solve(prob, True)
    
    timediff = float(solveres.time[:-1]) - float(solverespre.time[:-1]) - float(presolveinf.time[:-1])
    
    print(solveres.name)
    print("same result:", solveres.res == solverespre.res)
    print("solver time: " + solveres.time + "   solver time (pre): "+ solverespre.time)
    

air01.0.s.opb
same result: True
solver time: 0.15s   solver time (pre): 0.05s   diff: -0.36000000000000004
air01.0.u.opb
same result: True
solver time: 0.32s   solver time (pre): 0.13s   diff: -0.27
air02.0.s.opb
same result: True
solver time: 1.45s   solver time (pre): 1.32s   diff: -89.56
air02.0.u.opb
same result: True
solver time: 2.64s   solver time (pre): 0.71s   diff: -87.08
bm23.0.s.opb
same result: True
solver time: 0.01s   solver time (pre): 0.01s   diff: -0.01
bm23.0.u.opb
same result: True
solver time: 0.02s   solver time (pre): 0.02s   diff: -0.01
diamond.0.d.opb
same result: True
solver time: 0.00s   solver time (pre): 0.00s   diff: -0.01
p0040.0.s.opb
same result: True
solver time: 0.04s   solver time (pre): 0.00s   diff: 0.03
p0040.0.u.opb
same result: True
solver time: 0.04s   solver time (pre): 0.00s   diff: 0.03
p0291.0.s.opb
same result: True
solver time: 0.00s   solver time (pre): 0.00s   diff: -0.03
p0291.0.u.opb
same result: True
solver time: 0.01s   solver time 

In [179]:
prob = "cracpb1.0.s.opb"
solveres = solve(prob, False)
presolve(prob)
presolveres = solve(prob, True)
print(solveres.name)
print(solveres.res == presolveres.res)

cracpb1.0.s.opb
True


In [173]:
float(presolveres.time[:-1])

4.53

'36.94s'